In [2]:
import torch
from torch import Tensor
import numpy as np

In [ ]:
class Activation_functional (intput):
    
    def __init__(self,input,act):
        self.result = Tensor.new(input)
        self.kind   = act   # Relu, Sig, Relu' , sig'
        
    def __getitem__(self, index):
        return self.train_aug[index], self.train_target[index]
    
    def __len__(self):
        return len(self.train_target)
    
    
        
    def dRefu (self,input):
        result = 1-(input<0)
        
    def dSigmoid (self,input):
        result = 1-torch.pow(torch.tanh(input),2)
        
    
        
    def dTanh(self,input):
        result = 1-torch.tanh(x)*torch.tanh(x)
    
    return result

In [ ]:
class Seqeuntial (arch_list):
    
    def __init__(self,input,act):
        self.result = Tensor.new(input)
        self.kind   = act   # Relu, Sig, Relu' , sig'
        
    def __getitem__(self, index):
        return self.train_aug[index], self.train_target[index]
    
    def __len__(self):
        return len(self.train_target)
    
    def make_net (self,arch_list):
        nb = len(self.arch_list)
        
        for i in range (nb):
            check_name (arch_list[i])

# Auxiliary Classes

In [81]:
class Weight:
    """
    Initializes the synaptic weights
    
    inputs:
        layer_nb    :  the number of layer
        dim_in      :  the input dimension of fully connected layer
        dim_out     :  the output dimension of fully connected layer
        
    returns:
        Nothing
    """
    
    def __init__(self,layer_nb, dim_in, dim_out):
        self.shape = (dim_in , dim_out)
        self.name = layer_nb
        self.data = Tensor (dim_in,dim_out).normal_()

# it is a module in the net. it calls some specific parameter...   


class Linear:
    """
    An class that contains objects which only store layar's in/out connections dimension
    
    inputs:
        dim_in      :  the input dimension of fully connected layer
        dim_out     :  the output dimension of fully connected layer
        
    returns:
        Nothing 
    """
    
    def __init__(self,dim_in, dim_out):
        self.input = dim_in
        self.output = dim_out          

        
class Activation:
    """
    A class that specify the needed activation with respect to the following code conversion
        0: Relu(x)
        1: Tanh(x)
        2: Sigmoid(x)
    
    This class works as functional package of pytorch
    
    inputs:
        code        :  the code for each activation (0,1,2)
        input_tensor:  the input tensor
        
    returns:
        result      :  the output of requested activation function with the same shape as input tensor
    """
    
    def __init__(self, code, input):
        self.code = code
        self.result = torch.Tensor.new(input.shape)
    
    def Relu (self,input):
        result = input - (input<0);
        
    def Tanh (self,input): #WARNING!!!
        result = torch.tanh(input)
        
    def Sigmoid (self,input):
        result = 1.0/(1 + torch.exp(-input))

    
    def choose_activation (self, code):
        if self.code ==0:
            return self.Relu(input)
        elif self.code ==1:
            return self.Tanh(input)
        elif self.code ==2:
            return self.Sigmoid(input)
        else: raise UnknownCodeForActivation
        

# Network Class

In [82]:
class net:
    """
    The network class. It has the following methods:
        param      :  returns the parameter which is asked for. Not the data! The object... 
                        Data is accessible through object.data method)
        make_arch  :  makes the architecture of the network by taking a sequential list of [fc1,act1,fc2,act2,...]
    """
    
    def __init__ (self, seq):
        self.param_list =[]
        self.act_list = []
        self.make_arch(seq)
        
    def param (layer):
        return self.param_list[layer]
    
    def make_arch(self,seq):
        seq_len = len (seq)
        for layer in range (0,seq_len,2):
            # we are sure that seq[layer] is a FC object -> need to know in / out
            dim_in, dim_out = seq[layer].input , seq[layer].output 
            self.param_list.append ( Weight(layer, dim_in, dim_out) )#object (Activation / Linear) )   #... should be filled wrt seq type and inputs
            # activation recognition
            if seq[layer+1]=='relu':
                self.act_list.append(0)
            elif seq[layer+1]=='tanh':
                self.act_list.append(1)
            elif seq[layer+1]=='sig':
                self.act_list.append(2)
            else: raise UnknownActivation

# Draft, tests, and other stuff

In [80]:
seq = [Linear(4,7),'regu',Linear(7,4),'tanh']    
model = net (seq)

NameError: name 'UnknownActivation' is not defined

In [78]:
model.param_list[1].data


-0.3474  0.0254  1.1912 -0.4338
-0.1570  3.0292  0.7686  0.5892
-0.1642  0.1402  1.1387 -1.0072
-0.9680  1.3377 -1.0943 -1.2231
-2.2461 -0.8587 -1.4203  0.5847
 0.1913  1.0071  0.2806 -0.6999
-0.8223  1.6559  1.7871 -0.7000
[torch.FloatTensor of size 7x4]

In [69]:
a= Tensor(3,6)

In [70]:
a.shape

torch.Size([3, 6])